# 加载包 定义相关util函数

In [ ]:
nohup python3 run_classifier_mymodel.py \
--model_type roberta \
--task_name codesearch \
--do_train \
--do_eval \
--eval_all_checkpoints \
--train_file aspectj_train.txt \
--dev_file aspectj_val.txt \
--max_seq_length 256 \
--per_gpu_train_batch_size 32 \
--per_gpu_eval_batch_size 32 \
--learning_rate 1e-5 \
--num_train_epochs 10 \
--gradient_accumulation_steps 1 \
--overwrite_output_dir \
--data_dir /data/hdj/SourceFile/data/sourceFile_aspectj/train_valid \
--output_dir ./models/java/ghm  \
--model_name_or_path microsoft/codebert-base > ghm_10epoches_0506_summary_code_256.txt 2>&1 &

In [ ]:
nohup python3 run_classifier.py \
--model_type roberta \
--task_name codesearch \
--do_train \
--do_eval \
--eval_all_checkpoints \
--train_file aspectj_train.txt \
--dev_file aspectj_val.txt \
--max_seq_length 512 \
--per_gpu_train_batch_size 8 \
--per_gpu_eval_batch_size 8 \
--learning_rate 1e-5 \
--num_train_epochs 10 \
--gradient_accumulation_steps 1 \
--overwrite_output_dir \
--data_dir /data/hdj/SourceFile/data/sourceFile_aspectj/train_valid \
--output_dir ./models/java  \
--model_name_or_path microsoft/codebert-base > 0507_10ep_512_summary_code_256.txt 2>&1 &

In [ ]:
nohup python3 run_classifier.py \
--model_type roberta \
--model_name_or_path microsoft/codebert-base \
--task_name codesearch \
--do_predict \
--output_dir ../data/codesearch/test/aspectj_test/aspectj/ \
--data_dir /data/hdj/SourceFile/data/sourceFile_aspectj/train_valid/ \
--max_seq_length 512 \
--per_gpu_train_batch_size 256 \
--per_gpu_eval_batch_size 256 \
--learning_rate 1e-5 \
--num_train_epochs 8 \
--test_file aspectj_test.txt  \
--pred_model_dir /data/hdj/data/CodeBERT/codesearch/models/java/checkpoint-best \
--test_result_dir ./results/java/0507_10epoches_aspectj_summary_code_512.txt > 0507_10epoches_aspectj_summary_code_512.txt 2>&1 &

In [ ]:
nohup python3 main.py > res.txt 2>&1 &

In [1]:
import torch
from tqdm import tqdm, trange
import torch.nn as nn
from transformers import RobertaTokenizer, RobertaConfig, RobertaModel, AdamW
import transformers
import torch.nn.functional as F
from torch.nn import CrossEntropyLoss
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import numpy as np
import random
from sklearn import metrics
import torch.nn as nn
import argparse
import torch.distributed as dist
import torch.utils.data.distributed
import os
import numpy as np
from sklearn.metrics import f1_score
from utils import (compute_metrics, convert_examples_to_features,
                        output_modes, processors)
from torch.utils.data import (DataLoader, RandomSampler, SequentialSampler,
                              TensorDataset)
from transformers import (WEIGHTS_NAME, get_linear_schedule_with_warmup, AdamW,
                          RobertaConfig,
                          RobertaForSequenceClassification,
                          RobertaTokenizer)
from CodeBertModel import TextCNNClassifer_pair,TextCNNClassifer_pair_nofeature,TextCNNClassifer_cosloss
import os
# for i in range(58):
#     os.system('python3 run_classifier.py --model_type roberta --model_name_or_path microsoft/codebert-base --task_name codesearch --do_predict --output_dir ../data/codesearch/test/ --data_dir ../data/codesearch/test/ --max_seq_length 512 --per_gpu_train_batch_size 32 --per_gpu_eval_batch_size 32 --learning_rate 1e-5 --num_train_epochs 8 --test_file aspectj_'+str(i)+'.txt  --pred_model_dir ./models/java/checkpoint-best/ --test_result_dir ./results/java/batch_result_'+str(i)+'.txt')
from more_itertools import chunked
def calculate_same_value(labels_sorted, test_p_sorted, start_pos):
    i = start_pos
    num_same = 0
    num_p = 0
    while test_p_sorted[start_pos] == test_p_sorted[i]:
        num_same = num_same + 1
        if labels_sorted[i] ==1 : num_p = num_p + 1
        i = i + 1
        if i == len(labels_sorted ): break
    return num_p, num_same
def eval_mrr(test_p, labels):#在第二维相似度得分，真实标签
    test_p_sorted = test_p
    test_p_index = sorted(range(len(test_p_sorted)), key=lambda k: test_p_sorted[k], reverse=True)  # 降序排序
    test_p_sorted = sorted(test_p, reverse=True)

    labels_sorted = []
    for index in test_p_index:
        labels_sorted.append(labels[index])

    top_num = 10
    top10rank = 0
    for i in range(top_num):
        if (labels_sorted[i] == 1):
            '''
            num_p, num_s = calculate_same_value(labels_sorted, test_p_sorted, i)
            num_r = top_num - i
            if num_p > (num_s-num_r):
                top10rank = 1
                break
            v1 = perm(num_s-num_r, num_p)*perm(num_s-num_p,num_s-num_p)
            v2 = perm(num_s,num_s)
            top10rank = 1-(float)((float)(v1)/(float)(v2))
            if top10rank > 1: top10rank=1
            if top10rank!=top10rank: top10rank=1
            break

    return top10rank

    '''
            top10rank = 1
            break
    num_p, num_s = calculate_same_value(labels_sorted, test_p_sorted, 10)
    if (num_p >= 1): top10rank = 1  # 统计在第十位并列排名相同的文件中，是否含有相关文件

    MRRrank = 0.0
    for i in range(len(labels_sorted)):
        if (labels_sorted[i] == 1):
            MRRrank = MRRrank + float(1 / (i + 1))

    MAPrank = 0.0
    pos_num = 0
    for i in range(len(labels_sorted)):
        if (labels_sorted[i] == 1):
            pos_num = pos_num + 1
            MAPrank = MAPrank + float(pos_num / (i + 1))
    if pos_num==0:
        print('出现不存在pos的例子')
        pos_num=1
    MAPrank = float(MAPrank / pos_num)
    MRRrank = float(MRRrank / pos_num)

    return top10rank, MRRrank, MAPrank

def simple_accuracy(preds, labels):
#     print(type(preds),type(labels))
    return (preds == labels).mean()


def acc_and_f1(preds, labels):
    acc = simple_accuracy(preds, labels)
    f1 = f1_score(y_true=labels, y_pred=preds)
    return {
        "acc": acc,
        "f1": f1,
        "acc_and_f1": (acc + f1) / 2,
    }


def compute_metrics(preds, labels):
    assert len(preds) == len(labels)

    return acc_and_f1(preds, labels)
#设置种子，为了结果复现
def set_seed(args):
    random.seed(args.seed)
    np.random.seed(args.seed)
    torch.manual_seed(args.seed)

# 参数设置

In [2]:
class args(object):
    """A single set of features of data."""

    def __init__(self):
        self.model_type = 'roberta'
        self.output_dir='/data/hdj/data/CodeBERT/codesearch/models/java'
        self.test_result_dir='/data/hdj/tracking_buggy_files/joblib_memmap_eclipse/notebook_0607_test_2hard_2_data.txt'
        self.start_epoch=0
        self.config_name=''
        self.model_name_or_path=None
        self.task_name='codesearch'
        self.tokenizer_name=''
        self.model_name_or_path='microsoft/codebert-base'
        self.do_lower_case=True
        self.seed=42
        self.gradient_accumulation_steps=1
        self.weight_decay=0.0
        self.max_grad_norm=1.0
        self.learning_rate=2e-5#1e-6 5e-5
        self.adam_epsilon=1e-8
        self.warmup_steps=0
        self.max_steps=-1
        self.num_train_epochs=4
# class args(object):
#     """A single set of features of data."""

#     def __init__(self):
#         self.model_type = 'codesearch'
#         self.output_dir='/data/hdj/data/CodeBERT/codesearch/models/java'
#         self.test_result_dir='/data/hdj/data/CodeBERT/codesearch/results/java/test_v1_0414.txt'
#         self.start_epoch=0
#         self.num_train_epochs=1
#         self.model_type='roberta'
args=args()
#带权重的交叉熵
# weights=torch.tensor([0.2,0.8]).cuda()
# loss_fun=CrossEntropyLoss(weight=weights)
#不带权重的交叉熵
loss_fun=CrossEntropyLoss()
#设置种子 复现结果
set_seed(args)

In [3]:
pred=torch.tensor([[0.1,0.3],[0.2,0.5],[0.6,0.3]])
targets=torch.tensor([0,0,1])
loss_fun(pred,targets)

tensor(0.8356)

In [16]:

#加载GraphCodeBert模型代码
# from transformers import AutoModel
# tokenizer = AutoTokenizer.from_pretrained("microsoft/graphcodebert-base")
# model = AutoModel.from_pretrained("/home/hdj/graphcodebert-base")
#加载完毕

# print(model) 打印模型结构信息
# model.config 打印模型配置信息

In [3]:
ttype='train'
# test_file = "test_3hard_2_data.txt"#目前只使用一个例子来进行测试，后期增加数据量
# train_file='aspectj_train_oversample.txt'
train_file='train_3hard_1_train_data.txt'
# dev_file = "aspectj_val_oversample.txt"
max_seq_length=512
task='codesearch'
data_dir="/data/hdj/tracking_buggy_files/joblib_memmap_swt/one_vs_two/"
file_name = train_file.split('.')[0]
# dev_file = "train_3hard_1_valid_data.txt"
cached_features_file = os.path.join(data_dir, 'cached_{}_{}_{}_{}_{}'.format(
        ttype,
        file_name,
        list(filter(None, args.model_name_or_path.split('/'))).pop(),
        str(max_seq_length),
        str(task)))
cached_features_file

'/data/hdj/tracking_buggy_files/joblib_memmap_swt/one_vs_two/cached_train_train_3hard_1_train_data_codebert-base_256_codesearch'

# 加载并向量化数据

In [23]:
#加载数据并保存在cache里
def load_and_cache_examples(file_prefix,data_dir,train_file,dev_file,test_file, task, tokenizer,max_seq_length, ttype='train',need_feature=False):
    '''
        data_dir:数据目录
        train_file:训练文件
        dev_file:验证文件
        test_file:测试文件
        task:任务名 固定为codesearch
        tokenizer:分词器
        max_seq_length:最大序列长度 convert_examples_to_features使用到了
        ttype:类型 train dev test
        
        return：包装好的数据集 [all_input_ids, all_input_mask, all_segment_ids, all_label_ids]
    '''
    processor = processors[task]()#拿到CodesearchProcessor
    output_mode = output_modes[task]#输出模式 固定为 classification
    # if os.path.exists(cached_features_file):

    label_list = processor.get_labels()#固定为 ["0","1"]

    '''
        example结构：[InputExample(uid 'test-1',report,code,label)]
    '''
    if ttype == 'train':
        file_name = train_file.split('.')[0]
    elif ttype == 'dev':
        file_name = dev_file.split('.')[0]
    elif ttype == 'test':
        file_name = test_file.split('.')[0]
    cached_features_file = os.path.join(data_dir, 'cached_{}_{}_{}_{}_{}'.format(
        ttype,
        file_name,
        list(filter(None, args.model_name_or_path.split('/'))).pop(),
        str(max_seq_length),
        str(task)))
    try:
        print('直接加载cache文件')
        print('cached_features_file :',cached_features_file)
        features = torch.load(cached_features_file)
        if ttype == 'test':
            examples, instances = processor.get_test_examples(data_dir, test_file)
    except Exception as e:
        print(e)
        print('生成cache文件')
        examples=None
        if ttype == 'train':
            examples = processor.get_train_examples(data_dir, train_file)
        elif ttype == 'dev':
            examples = processor.get_dev_examples(data_dir=data_dir, dev_file= dev_file)
        elif ttype == 'test':
            #如果是test的话，instances就是[每一行是一条代测试的数据]
            examples, instances = processor.get_test_examples(data_dir, test_file)

        '''
            example:[[uid,report,code,label],[]...]
            label_list:['0','1']
            max_seq_length:200
            tokenizer:分词器
            output_mode:输出模式 固定为 classification
            cls_token:用来分类的token     '<s>', 0, 
            sep_token:用来分割语句的token '</s>', 2

            return：[  [       ( input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids,
                              label_id=label_id)
                        ]
                    ]
                    instances：[["0","5222","i am a noy","public class"],[],,]
        '''
    #     features = convert_examples_to_features(examples, label_list, max_seq_length, tokenizer, output_mode,
    #                                             cls_token=tokenizer.cls_token,
    #                                             sep_token=tokenizer.sep_token,
    #                                            )
        features =convert_examples_to_features(examples, label_list, max_seq_length, tokenizer, output_mode,
                                                    cls_token_at_end=bool(args.model_type in ['xlnet']),
                                                    # xlnet has a cls token at the end
                                                    cls_token=tokenizer.cls_token,
                                                    sep_token=tokenizer.sep_token,
                                                    cls_token_segment_id=2 if args.model_type in ['xlnet'] else 1,
                                                    pad_on_left=bool(args.model_type in ['xlnet']),
                                                    # pad on the left for xlnet
                                                    pad_token_segment_id=4 if args.model_type in ['xlnet'] else 0)
        torch.save(features, cached_features_file)
    # Convert to Tensors and build dataset
    all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
    all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_segment_ids = torch.tensor([f.segment_ids for f in features], dtype=torch.long)
    
    if output_mode == "classification":
        all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.long)
    if need_feature:
        feature=torch.tensor([f.feature for f in features],dtype=torch.float16)
        print('human_feature :',feature)
        dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids,feature)
    else:
        dataset = TensorDataset(all_input_ids, all_input_mask, all_segment_ids, all_label_ids)
    if (ttype == 'test'):
        return dataset, instances
    else:
        return dataset
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base",do_lower_case=False)

In [33]:
nl_len=[]
code_len=[]
nl_content=set()
code_content=set()
with open('/data/hdj/tracking_buggy_files/joblib_memmap_eclipse/one_vs_two/train_2hard_1_train_source_nomethod.txt','r',encoding='utf-8') as f_in:
    for line in f_in:
        contents=line.split('<CODESPLIT>')
        nl_content.add(contents[3])
        code_content.add(contents[4])
for con in nl_content:
    nl_tokens=tokenizer.tokenize(con)
    nl_len.append(len(nl_tokens))
for con in code_content:
    nl_tokens=tokenizer.tokenize(con)
    code_len.append(len(nl_tokens))

In [34]:
len(nl_len),len(code_len)

(4267, 32399)

In [35]:
# summary_len=[]
def statis(sourceLen,name):#list
    a = np.array(sourceLen)
    print(name,'code 的 10%', np.percentile(a, 10))
    print('20 :', np.percentile(a, 20))
    print('30 :', np.percentile(a, 30))
    print('40 :', np.percentile(a, 40))
    print('50 :', np.median(a))
    print('60 :', np.percentile(a, 60))
    print('70 :', np.percentile(a, 70))
    print('80 :', np.percentile(a, 80))
    print('90 :', np.percentile(a, 90))
statis(nl_len,'nl_len')
statis(code_len,'code_len')

nl_len code 的 10% 13.0
20 : 23.0
30 : 40.0
40 : 53.0
50 : 67.0
60 : 84.0
70 : 112.0
80 : 161.0
90 : 362.4000000000001
code_len code 的 10% 30.0
20 : 52.0
30 : 77.0
40 : 105.0
50 : 141.0
60 : 187.0
70 : 250.0
80 : 348.0
90 : 547.0


In [3]:
def _truncate_seq_pair(tokens_a, tokens_b, max_length):
    """Truncates a sequence pair in place to the maximum length."""

    # This is a simple heuristic which will always truncate the longer sequence
    # one token at a time. This makes more sense than truncating an equal percent
    # of tokens from each, since if one sequence is very short then each token
    # that's truncated likely contains more information than a longer sequence.
    while True:
        total_length = len(tokens_a) + len(tokens_b)
        if total_length <= max_length:
            break
        if len(tokens_a) > len(tokens_b):
            tokens_a.pop()
        else:
            tokens_b.pop()
def convert_examples_to_features_hdj(examples,label_list,nl_max_len,max_len,tokenizer):
    #code
    label_map = {label: i for i, label in enumerate(label_list)}

    features = []
    for (ex_index, example) in enumerate(examples):
        if ex_index % 10000 == 0:
            print("Writing example %d of %d" % (ex_index, len(examples)))
            
      
       
        nl=example.text_a
        nl_tokens=tokenizer.tokenize(nl)
#         print('nl token len :',len(nl_tokens))
#         print('nl :',nl_tokens)
        nl_tokens=nl_tokens[:nl_max_len]     
        code=example.text_b
        code_tokens=tokenizer.tokenize(code)
#         print('code token len :',len(code_tokens))
#         print('code :',code_tokens)
        _truncate_seq_pair(nl_tokens, code_tokens, max_len - 3)
        tokens = [tokenizer.cls_token]+nl_tokens + [tokenizer.sep_token]+code_tokens+ [tokenizer.sep_token]
       
        input_ids = tokenizer.convert_tokens_to_ids(tokens)
        input_mask = [1] * len(input_ids)
        label_id = label_map[example.label]
        padding_length = max_len - len(input_ids)
        input_ids = input_ids + ([tokenizer.pad_token_id] * padding_length)
        input_mask = input_mask + ([0] * padding_length)
#         print('input_ids :',input_ids)
#         print('input_mask :',input_mask)
       
        features.append(
            InputFeatures_hdj(input_ids=input_ids,
                          input_mask=input_mask,
                          label_id=label_id))
    return features

In [4]:
#加载数据并保存在cache里
class InputFeatures_hdj(object):
    """A single training/test features for a example."""
    def __init__(self,
                 input_ids,
                 input_mask,
                 label_id,

    ):
        self.input_ids = input_ids
        self.input_mask=input_mask
        self.label_id=label_id
class InputFeatures_hdj_split(object):
    """A single training/test features for a example."""
    def __init__(self,
                 code_tokens,
                 code_ids,
                 code_mask,
                 nl_tokens,
                 nl_ids,
                 nl_mask,
                 label_id,

    ):
        self.code_tokens = code_tokens
        self.code_ids = code_ids
        self.code_mask=code_mask
        self.nl_tokens = nl_tokens
        self.nl_ids = nl_ids
        self.nl_mask=nl_mask
        self.label_id=label_id
def convert_examples_to_features_hdj_split(examples,label_list,nl_max_len,max_len,tokenizer):
    #code
#     label_map = {'0':-1,'1':1}
    label_map = {label: i for i, label in enumerate(label_list)}
    features = []
    for (ex_index, example) in enumerate(examples):
        if ex_index % 10000 == 0:
            print("Writing example %d of %d" % (ex_index, len(examples)))
            
        code=example.text_b
        code_tokens=tokenizer.tokenize(code)[:max_len-2]
        code_tokens =[tokenizer.cls_token]+code_tokens+[tokenizer.sep_token]
        code_ids =  tokenizer.convert_tokens_to_ids(code_tokens)
        code_input_mask = [1] * len(code_ids)
        padding_length = max_len - len(code_ids)
        code_ids+=[tokenizer.pad_token_id]*padding_length
        code_input_mask+=[0]*padding_length

        nl=example.text_a
        nl_tokens=tokenizer.tokenize(nl)[:max_len-2]
        nl_tokens =[tokenizer.cls_token]+nl_tokens+[tokenizer.sep_token]
        nl_ids =  tokenizer.convert_tokens_to_ids(nl_tokens)
        nl_input_mask = [1] * len(nl_ids)
        padding_length = max_len - len(nl_ids)
        nl_ids+=[tokenizer.pad_token_id]*padding_length
        nl_input_mask+=[0]*padding_length
        label_id = label_map[example.label]
#         print('code_tokens :',code_tokens)
#         print('code_ids :',code_ids)
#         print('code_mask :',code_input_mask)
        features.append(
            InputFeatures_hdj_split(code_tokens = code_tokens,
                            code_ids = code_ids,
                            code_mask=code_input_mask,
                            nl_tokens = nl_tokens,
                            nl_ids = nl_ids,
                            nl_mask=nl_input_mask,
                            label_id=label_id))
    return features
def load_and_cache_examples_hdj(file_prefix,data_dir,train_file,dev_file,test_file, task, tokenizer,max_seq_length, ttype='train',need_feature=False):
    '''
        data_dir:数据目录
        train_file:训练文件
        dev_file:验证文件
        test_file:测试文件
        task:任务名 固定为codesearch
        tokenizer:分词器
        max_seq_length:最大序列长度 convert_examples_to_features使用到了
        ttype:类型 train dev test
        
        return：包装好的数据集 [all_input_ids, all_input_mask, all_segment_ids, all_label_ids]
    '''
    processor = processors[task]()#拿到CodesearchProcessor
    output_mode = output_modes[task]#输出模式 固定为 classification
    # if os.path.exists(cached_features_file):

    label_list = processor.get_labels()#固定为 ["0","1"]

    '''
        example结构：[InputExample(uid 'test-1',report,code,label)]
    '''
    if ttype == 'train':
        file_name = train_file.split('.')[0]
    elif ttype == 'dev':
        file_name = dev_file.split('.')[0]
    elif ttype == 'test':
        file_name = test_file.split('.')[0]
    cached_features_file = os.path.join(data_dir, 'cached_{}_{}_{}_{}_{}'.format(
        ttype,
        file_name,
        list(filter(None, args.model_name_or_path.split('/'))).pop(),
        str(max_seq_length),
        str(task)))
    try:
        print('直接加载cache文件')
        print('cached_features_file :',cached_features_file)
        features = torch.load(cached_features_file)
        if ttype == 'test':
            examples, instances = processor.get_test_examples(data_dir, test_file)
    except Exception as e:
        print(e)
        print('生成cache文件')
        examples=None
        if ttype == 'train':
            examples = processor.get_train_examples(data_dir, train_file)
        elif ttype == 'dev':
            examples = processor.get_dev_examples(data_dir=data_dir, dev_file= dev_file)
        elif ttype == 'test':
            #如果是test的话，instances就是[每一行是一条代测试的数据]
            examples, instances = processor.get_test_examples(data_dir, test_file)

        '''
            example:[[uid,report,code,label],[]...]
            label_list:['0','1']
            max_seq_length:200
            tokenizer:分词器
            output_mode:输出模式 固定为 classification
            cls_token:用来分类的token     '<s>', 0, 
            sep_token:用来分割语句的token '</s>', 2

            return：[  [       ( input_ids=input_ids,
                              input_mask=input_mask,
                              segment_ids=segment_ids,
                              label_id=label_id)
                        ]
                    ]
                    instances：[["0","5222","i am a noy","public class"],[],,]
        '''
    #     features = convert_examples_to_features(examples, label_list, max_seq_length, tokenizer, output_mode,
    #                                             cls_token=tokenizer.cls_token,
    #                                             sep_token=tokenizer.sep_token,
    #                                            )
        features =convert_examples_to_features_hdj_split(examples, label_list, nl_max_len,max_seq_length, tokenizer)
        torch.save(features, cached_features_file)
    # Convert to Tensors and build dataset
#     all_input_ids = torch.tensor([f.input_ids for f in features], dtype=torch.long)
#     all_input_mask = torch.tensor([f.input_mask for f in features], dtype=torch.long)
    all_code_input_ids = torch.tensor([f.code_ids for f in features], dtype=torch.long)
    all_code_input_mask = torch.tensor([f.code_mask for f in features], dtype=torch.long)
    all_nl_input_ids = torch.tensor([f.nl_ids for f in features], dtype=torch.long)
    all_nl_input_mask = torch.tensor([f.nl_mask for f in features], dtype=torch.long)
    if output_mode == "classification":
        all_label_ids = torch.tensor([f.label_id for f in features], dtype=torch.long)
    if need_feature:
        feature=torch.tensor([f.feature for f in features],dtype=torch.float16)
        print('human_feature :',feature)
        dataset = TensorDataset(all_code_input_ids, all_code_input_mask, all_nl_input_ids, all_nl_input_mask,all_label_ids,feature)
    else:
#         dataset = TensorDataset(all_input_ids, all_input_mask, all_label_ids)
        dataset = TensorDataset(all_code_input_ids, all_code_input_mask, all_nl_input_ids,all_nl_input_mask,all_label_ids)
    if (ttype == 'test'):
        return dataset, instances
    else:
        return dataset
tokenizer = RobertaTokenizer.from_pretrained("microsoft/codebert-base")#,do_lower_case=

In [5]:
project='eclipse'
file_prefix='/data/hdj/tracking_buggy_files/'+project+'/'+project
data_dir = "/data/hdj/tracking_buggy_files/joblib_memmap_"+project+"/one_vs_two/"
test_dir="/data/hdj/tracking_buggy_files/joblib_memmap_"+project+"/one_vs_two/" #注意目前测试集的目录是单独的，更换测试数据需要更改
# train_file = "aspectj_train_small.txt"
# dev_file = "aspectj_val_small.txt"
test_file = "test_2hard_2_2tok5meth_data.txt"#目前只使用一个例子来进行测试，后期增加数据量
# train_file='aspectj_train_oversample.txt'
train_file='train_2hard_1_train_source_nomethod_less.txt'
# dev_file = "aspectj_val_oversample.txt"
dev_file = "train_2hard_1_val_source_nomethod_less.txt"
task_name = "codesearch"
max_seq_length = 512#关键参数设置 整个序列的最大长度
need_feature=False
nl_max_len=128
train_set = load_and_cache_examples_hdj(file_prefix,data_dir,train_file,dev_file,test_file,task_name,tokenizer,max_seq_length,'train')
val_set=load_and_cache_examples_hdj(file_prefix,data_dir,train_file,dev_file,test_file,task_name,tokenizer,max_seq_length,'dev')

直接加载cache文件
cached_features_file : /data/hdj/tracking_buggy_files/joblib_memmap_eclipse/one_vs_two/cached_train_train_2hard_1_train_source_nomethod_less_codebert-base_512_codesearch
直接加载cache文件
cached_features_file : /data/hdj/tracking_buggy_files/joblib_memmap_eclipse/one_vs_two/cached_dev_train_2hard_1_val_source_nomethod_less_codebert-base_512_codesearch


In [15]:
# val_set.tensors

In [40]:
import gc
gc.collect()
torch.cuda.empty_cache()

In [41]:
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 1            |        cudaMalloc retries: 1         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |   22782 MB |   22824 MB |   36076 MB |   13294 MB |
|       from large pool |   22778 MB |   22820 MB |   36070 MB |   13292 MB |
|       from small pool |       3 MB |       3 MB |       5 MB |       1 MB |
|---------------------------------------------------------------------------|
| Active memory         |   22782 MB |   22824 MB |   36076 MB |   13294 MB |
|       from large pool |   22778 MB |   22820 MB |   36070 MB |

In [25]:
#数据集的配置参数
project='eclipse'
file_prefix='/data/hdj/tracking_buggy_files/'+project+'/'+project
data_dir = "/data/hdj/tracking_buggy_files/joblib_memmap_"+project+"/one_vs_two/"
test_dir="/data/hdj/tracking_buggy_files/joblib_memmap_"+project+"/one_vs_two/" #注意目前测试集的目录是单独的，更换测试数据需要更改
# train_file = "aspectj_train_small.txt"
# dev_file = "aspectj_val_small.txt"
test_file = "test_2hard_2_2tok5meth_data.txt"#目前只使用一个例子来进行测试，后期增加数据量
# train_file='aspectj_train_oversample.txt'
train_file='train_2hard_1_train_source_nomethod_less.txt'
# dev_file = "aspectj_val_oversample.txt"
dev_file = "train_2hard_1_val_source_nomethod_less.txt"
task_name = "codesearch"
max_seq_length = 256#关键参数设置 整个序列的最大长度
need_feature=False
#加载数据集
#这里load_and_cache_examples 是没有将数据进行缓存的，和codeBert里的同名函数意义不一样，所以统一按照每次都读取数据这样的模式来处理
train_set = load_and_cache_examples(file_prefix,data_dir,train_file,dev_file,test_file,task_name,tokenizer,max_seq_length,'train')
val_set = load_and_cache_examples(file_prefix,data_dir,train_file,dev_file,test_file,task_name,tokenizer,max_seq_length,'dev')
# # test_set, instances=load_and_cache_examples(test_dir,train_file,dev_file,test_file,task_name,tokenizer,max_seq_length,'test')


直接加载cache文件
cached_features_file : /data/hdj/tracking_buggy_files/joblib_memmap_eclipse/one_vs_two/cached_train_train_2hard_1_train_source_nomethod_codebert-base_256_codesearch
[Errno 2] No such file or directory: '/data/hdj/tracking_buggy_files/joblib_memmap_eclipse/one_vs_two/cached_train_train_2hard_1_train_source_nomethod_codebert-base_256_codesearch'
生成cache文件
直接加载cache文件
cached_features_file : /data/hdj/tracking_buggy_files/joblib_memmap_eclipse/one_vs_two/cached_dev_train_2hard_1_val_source_nomethod_codebert-base_256_codesearch
[Errno 2] No such file or directory: '/data/hdj/tracking_buggy_files/joblib_memmap_eclipse/one_vs_two/cached_dev_train_2hard_1_val_source_nomethod_codebert-base_256_codesearch'
生成cache文件


In [6]:
# #将数据进行batch_size设置
batch_size = 8 #关键参数
training_loader = DataLoader(dataset=train_set, batch_size=batch_size, shuffle=True, num_workers=4, drop_last=False)
val_loader = DataLoader(dataset=val_set, batch_size=batch_size, shuffle=False, num_workers=4, drop_last=False)
#测试集顺序不应该打乱，不然后期就对不上了 shuffle的作用是在枚举的时候才会改变顺序 
# testing_loader = DataLoader(dataset=test_set, batch_size=batch_size, shuffle=False, num_workers=2, drop_last=False)

In [36]:
# batch_size = 64 #关键参数
# val_loader

In [41]:
#目前train val数据集比较不均衡 test数据集比较均衡
print('训练集比例',sum(list(training_loader.dataset.tensors[2])),len(training_loader.dataset.tensors[2]),sum(list(training_loader.dataset.tensors[2]))/len(training_loader.dataset.tensors[2]))
print('验证集比例',sum(list(val_loader.dataset.tensors[2])),len(val_loader.dataset.tensors[2]),sum(list(val_loader.dataset.tensors[2]))/len(val_loader.dataset.tensors[2]))
# print('测试集比例',sum(list(testing_loader.dataset.tensors[3])),len(testing_loader.dataset.tensors[3]))

训练集比例 tensor(-52621) 148299 tensor(-0.3548)
验证集比例 tensor(-5974) 16450 tensor(-0.3632)


In [ ]:
#数据加载 设计好 小数据 均匀一些 方便切换大数据 
#方便添加多个训练集的数据 进行数据拆分重组
#添加数据统计模块 数据均衡度如何 
#定义好数据的格式 比例
#各个函数的参数类型 格式 方便查找 不然又忘了
#train val test

#训练采用codeBERT GraphCodeBert 或者使用cnn/ast + codeBERT/GraphCodeBert
#模型1 [codeBert]
#模型2 [GraphCodeBert]
#模型3 [cnn,codeBert]
#模型4 [cnn,GraphCodeBert]
#模型5 [ast,codeBert]
#模型6 [ast,GraphCodeBert]
#模型7 [cnn,ast,codeBert/GraphCodeBert]
#模型8 [cnn,ast,codeBert/GraphCodeBert]
#训练采用cross_entroy作为损失函数/复杂一点的使用triplet loss,val时就采用 f1、map mrr top来衡量 测试也是使用一样的指标


# 模型加载和信息

In [7]:
#模型1的实现
device = 'cuda' if torch.cuda.is_available() else 'cpu'
google_v=False #当为true时，使用RobertClassification模型
num_labels=2
class Model(nn.Module):   
    def __init__(self, encoder):
        super(Model, self).__init__()
        self.encoder = encoder
      
    def forward(self, code_inputs=None, nl_inputs=None): 
        if code_inputs is not None:
            return self.encoder(code_inputs,attention_mask=code_inputs.ne(1))[1]
        else:
            return self.encoder(nl_inputs,attention_mask=nl_inputs.ne(1))[1]
MODEL_CLASSES = {'roberta': (RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)}
config_class, model_class, tokenizer_class = MODEL_CLASSES[args.model_type]
config = config_class.from_pretrained(args.config_name if args.config_name else args.model_name_or_path,
                                     num_labels=num_labels, finetuning_task=args.task_name)
# if google_v:
#     model = model_class.from_pretrained(args.model_name_or_path, from_tf=bool('.ckpt' in args.model_name_or_path),
#                                     config=config)
#     print('使用RobertClassification 模型')
# else:
#     if need_feature:
#         model = TextCNNClassifer_pair()
#     else:
#         model = TextCNNClassifer_pair_nofeature()
# #         model=TextCNNClassifer_cosloss()
#     print('使用TextCNNClassifier模型')
checkpoint_prefix = 'checkpoint-best-mrr/model.bin'
output_dir = os.path.join('/data/hdj/data/CodeBERT/Siamese-model/saved_models/java', '{}'.format(checkpoint_prefix)) 
print(output_dir)
model = RobertaModel.from_pretrained(args.model_name_or_path)    
model=Model(model)
model.load_state_dict(torch.load(output_dir),strict=False)      
# model.to(args.device)
_ = model.to(device)






/data/hdj/data/CodeBERT/Siamese-model/saved_models/java/checkpoint-best-mrr/model.bin


In [8]:
if google_v or True:
    #Google: 使用的优化器和迭代器
    no_decay = ['bias', 'LayerNorm.weight']
    optimizer_grouped_parameters = [
        {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
         'weight_decay': args.weight_decay},
        {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
    ]
    optimizer = AdamW(optimizer_grouped_parameters, lr=args.learning_rate, eps=args.adam_epsilon)
    #Google
else:
#     hdj: textCNNClassifier 使用的优化器和计划器
    optimizer = torch.optim.Adam(model.parameters(), lr=1e-05, betas=(0.9, 0.999))
    scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=20, gamma=0.5)
#     hdj

In [9]:
from tensorboardX import SummaryWriter
tb_writer = SummaryWriter()

# 使用cosloss的代码

In [10]:
#开始验证模型的有效性 把模型预测结果连同数据一起保存起来 然后再去判断top mrr mAP
#流程 1.训练过程中 每个10轮去在val集上验证 比较f1值是否优于之前，是就保存结果到best，2.每次保存上一次验证的结果 3.最后再在test集上进行测试

#check 词表是否与之前model有很大的不同 如果有那么该怎么改善，之前embedding矩阵是否就作废了
def evaluate(args, model,eval_loader,instances,tokenizer, checkpoint=None, prefix="", mode='dev'):
    '''
        args:模型训练时关键参数
        model：待验证的模型
        eval_loader:待验证的数据集 DataLoader 类型的参数
        instances: 若为test模式，则将原始数据输入进来，然后将预测结果一起保存，等待后面计算TOP MAP MRR使用
        tokenizer:分词器
        checkpoint: 检查点
        prefix:前缀
        mode: 模式 分为 dev 和 test（需要保存数据）
        
        return: dict{'acc':,'f1':,'acc_f1'} 主要关注f1的值 越大说明正例的预测越准
    '''
    #待返回的验证结果
    results={}
    eval_loss = 0.0 #验证损失累加
#     nb_eval_steps = len(test_set)/batch_size #验证批次
    preds = None #保存预测的值
    out_label_ids = None #保存真实标签target
    print('开始验证。。。')
    for _, data in enumerate(eval_loader, 0):#start=0 默认就是0
        with torch.no_grad():
            code_input_ids = data[0].to(device, dtype=torch.long).cuda(non_blocking=True)
            code_input_mask = data[1].to(device, dtype=torch.long).cuda(non_blocking=True)
            nl_input_ids= data[2].to(device, dtype=torch.long).cuda(non_blocking=True)
            nl_input_mask= data[3].to(device, dtype=torch.long).cuda(non_blocking=True)
        #     token_type_ids = data[2].to(device, dtype=torch.long).cuda(non_blocking=True) 因为模型不支持2维设置 所以不再输入这个参数
            targets = data[4].to(device, dtype=torch.long).cuda(non_blocking=True)
            if need_feature:
                feature = data[4].to(device, dtype=torch.long).cuda(non_blocking=True)
            if google_v:
                pred = model(input_ids=ids, attention_mask=mask,token_type_ids=None, labels=targets)
                loss=pred[0]
            else:
                if need_feature:
                    pred = model(ids, mask,feature)#输出是[batch_size,2]
                else:
                    #get code and nl vectors
                    code_vec = model(code_inputs=code_input_ids)
                    nl_vec = model(nl_inputs=nl_input_ids)
#                     loss,code_vec,nl_vec = model(code_input_ids, code_input_mask,nl_input_ids,nl_input_mask,targets)
#                     print(loss,code_vec,nl_vec)
            # print(pred, target)
#             loss=floss(pred,targets)
#             loss = loss_fun(pred, targets)
#             pred_choice = pred.max(1)[1]
#             correct = pred_choice.eq(targets).cpu().sum()
#             metrics = compute_metrics(pred_choice.cpu().numpy(), targets.cpu().numpy())
#             print('[',epoch,': ',_,'/',nb_eval_steps,'] ',blue('val')," loss :%.4f" % loss.item(),' , accuracy: ',correct.item() / float(batch_size) ,'true nums:',sum(targets.cpu().numpy()),' ration :',sum(targets.cpu().numpy())/len(targets),"acc :",metrics['acc']," f1 :",metrics['f1'])
#             loss_list_val.append(loss.item())  
            global eval_step
            tb_writer.add_scalar('eval_loss', loss.item(), eval_step)
#             tb_writer.add_scalar('eval_acc', correct.item() / float(batch_size), eval_step)
#             tb_writer.add_scalar('eval_f1', metrics['f1'], eval_step)
            eval_step+=1
#             print(eval_step)
            eval_loss += loss.item()
            if(_%100==0):
                print(_,'/',len(eval_loader),loss)
#         nb_eval_steps += 1
        #将preds值和标签进行保存
#         if preds is None:
#             if google_v:
#                 preds = pred[1].detach().cpu()
#             else:
#                 preds=pred.detach().cpu()
#             out_label_ids = targets.detach().cpu()
#         else:
#             if google_v:
#                 preds = np.append(preds, pred[1].detach().cpu().numpy(), axis=0)
#             else:    
#                 preds = np.append(preds, pred.detach().cpu().numpy(), axis=0)
#             out_label_ids = np.append(out_label_ids, targets.detach().cpu().numpy(), axis=0)       
#     print('验证结束，开始计算指标',eval_loss)
#     #计算指标，并更新结果
 
#     #ndarra转换为tensor类型
#     preds=torch.tensor(preds)
#     out_label_ids=torch.tensor(out_label_ids)
# #     print(type(preds),preds.shape,preds)
# #     print(type(out_label_ids),out_label_ids.shape,out_label_ids)    
#     pred_choice = preds.max(1)[1]#[batch_size,2]
#     correct = pred_choice.eq(out_label_ids).cpu().sum() #eq需要tensor类型的数据
#     #但是compute_metrics 需要numpy的数据类型 真烦人
#     metrics = compute_metrics(pred_choice.numpy(), out_label_ids.numpy())
#     results.update(metrics)
    #计算完成
    
    #下面分dev 和 test 模式来完成数据模型的保存工作
#     if(mode=='dev'):
#         #这里只是把result字典里acc和f1给保存到文件
#         output_eval_file = os.path.join(args.output_dir, "eval_results.txt")
#         with open(output_eval_file, "a+") as writer:
#             print("***** Eval results {} *****".format(prefix))
#             writer.write('evaluate %s\n' % checkpoint)
#             for key in sorted(results.keys()):
#                 writer.write("%s = %s\n" % (key, str(results[key])))
#     elif(mode=='test'):
#         #test结果目录 将test数据连同预测结果进行保存
#         if(instances==None or len(instances)==0):
#             print('输入的测试数据有问题 为None 或者 长度为0')
#         #如果目录不存在就创建
#         output_test_file = args.test_result_dir
#         output_dir = os.path.dirname(output_test_file)
#         if not os.path.exists(output_dir):
#             os.makedirs(output_dir)
#         #检验创建目录完成
#         with open(output_test_file, "w") as writer:
#             print("***** Output test results *****")
#             preds = F.softmax(torch.tensor(preds))
#             all_logits = preds.tolist()
#             for i, logit in tqdm(enumerate(all_logits), desc='Testing'):#desc是进度条的标题
#                 #instances即代表test集里每一行数据的值
#                 instance_rep = '<CODESPLIT>'.join([item.encode('ascii', 'ignore').decode('ascii') for item in instances[i]])
#                 writer.write(instance_rep + '<CODESPLIT>' + '<CODESPLIT>'.join([str(l) for l in logit]) + '\n')
#             #打印验证的结果
#             for key in sorted(results.keys()):
#                 print("%s = %s" % (key, str(results[key])))
#     return results
    return None

In [11]:
#模型应该保存效果最好的那个，而不是训练到最后的那个模型
print("training!!!")
blue = lambda x: '\033[94m' + x + '\033[0m'
epoch=1
best_f1=0.0 #保存最好的f1的模型
model.zero_grad()
set_seed(args)  # Added here for reproductibility (even between python 2 and 3)
#这里就是在指定 一共训练多少Epoch [start_epoch,num_train_epcchs]
t_total = len(training_loader) // args.gradient_accumulation_steps * args.num_train_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, args.warmup_steps, t_total)
global_step=0
eval_step=0
embedding_loss=torch.nn.CosineEmbeddingLoss(margin=0.5, size_average=None, reduce=None, reduction='mean')
train_iterator = trange(args.start_epoch, int(args.num_train_epochs), desc="Epoch")
model.train()
for idx, p in enumerate(train_iterator):
    num_batch = len(train_set) / batch_size #总共训练次数
    for _, data in enumerate(training_loader, 0):#start=0 默认就是0
        code_input_ids = data[0].to(device, dtype=torch.long).cuda(non_blocking=True)
        code_input_mask = data[1].to(device, dtype=torch.long).cuda(non_blocking=True)
        nl_input_ids= data[2].to(device, dtype=torch.long).cuda(non_blocking=True)
        nl_input_mask= data[3].to(device, dtype=torch.long).cuda(non_blocking=True)
    #     token_type_ids = data[2].to(device, dtype=torch.long).cuda(non_blocking=True) 因为模型不支持2维设置 所以不再输入这个参数
        targets = data[4].to(device, dtype=torch.long).cuda(non_blocking=True)
        if need_feature:
            feature=data[4].to(device, dtype=torch.long).cuda(non_blocking=True)
#         print('feature shape: ',feature.shape)
#         print(feature)
        if google_v:
#             outputs = model(ids, mask, token_type_ids)
            outputs = model(input_ids=ids, attention_mask=mask,token_type_ids=None, labels=targets)
            loss=outputs[0]#google loss
#             print(outputs)
#             loss_hdj = loss_fun(outputs[1], targets)
#             print('google loss ',loss,' loss_hdj :',loss_hdj)
            pred_choice = outputs[1].max(1)[1]
        else:
            if need_feature:
                outputs = model(ids, mask,feature)#输出是[batch_size,2]
            else:
                code_vec = model(code_inputs=code_input_ids)
                nl_vec = model(nl_inputs=nl_input_ids)
                loss=embedding_loss(code_vec,nl_vec,targets)
#                 code_vec,nl_vec = model(code_input_ids, code_input_mask,nl_input_ids,nl_input_mask)
            #使用floss
#             loss=floss(outputs,targets)
            #使用floss
            #使用ghmloss
#             ghm_target = targets.view(-1, 1)
#             ghm_target=ghm_target.cpu()
#             ghm_target = torch.LongTensor(ghm_target)
#             # print("before target :",targets)
#             ghm_target = torch.zeros(len(ghm_target), 2).scatter_(1, ghm_target, 1)
#             outputs=outputs.cpu()
#             loss=ghm(outputs,ghm_target)
            #使用ghmloss
            # print(outputs)
            
            #交叉熵loss
#             loss = loss_fun(outputs, targets)
            #交叉熵loss
    
    #         print('查看target和ouput的类型 检查max函数返回的东西是否正确',type(outputs),type(targets))
        #     print(outputs)
            #取outputs二维中最大维的下标
#             pred_choice = outputs.max(1)[1]
        if args.gradient_accumulation_steps > 1:
             loss = loss / args.gradient_accumulation_steps
                
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
        
#         targets=targets.cpu()
#         correct = pred_choice.eq(targets).cpu().sum()
#         metrics = compute_metrics(pred_choice.cpu().numpy(), targets.cpu().numpy())
        if _ %100 == 0:
            print("loss :%.4f" % loss.item())
#             print('[',epoch,':',_,'/',num_batch,']',"loss :%.4f" % loss.item(),' , accuracy: ',correct.item() / float(batch_size) ,'true nums:',sum(targets.cpu().numpy()),' ration :',sum(targets.cpu().numpy())/len(targets),"acc :",metrics['acc']," f1 :",metrics['f1'])
#         loss_list.append(loss.item())
#         acc_list.append(correct.item() / float(batch_size))
        if(_+1)%args.gradient_accumulation_steps == 0:
            global_step+=1
            optimizer.step()
            scheduler.step()
            model.zero_grad()
            tb_writer.add_scalar('train_loss', loss.item(), global_step)
#             tb_writer.add_scalar('train_acc', correct.item() / float(batch_size), global_step)
#             tb_writer.add_scalar('train_f1', metrics['f1'], global_step)
    #下面是在训练过程中进行验证 每一轮都会验证一次，保存模型最好的 和最近一次的模型
    results = evaluate(args, model,val_loader,None ,tokenizer, checkpoint=str(args.start_epoch + idx),mode='dev')
#     print('验证结果 ：',results)
    #torch.save(model, '%s/cls_model_%d.pth' % ('models', epoch))
    #./models/java/checkpoint-last
    last_output_dir = os.path.join(args.output_dir, 'checkpoint-last')
    if not os.path.exists(last_output_dir):
        os.makedirs(last_output_dir)
#     model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
#     model_to_save.save_pretrained(last_output_dir)
    torch.save(model, '%s/cls_model_%d.pth' % (last_output_dir, epoch))
    print("Saving model checkpoint to %s", last_output_dir)
    #保存idx_file 文件 第几轮训练的记录
    idx_file = os.path.join(last_output_dir, 'idx_file.txt')
    with open(idx_file, 'w', encoding='utf-8') as idxf:
        idxf.write(str(args.start_epoch + idx) + '\n')
    #保存idx_file完成
    #保存优化器和计划器
    torch.save(optimizer.state_dict(), os.path.join(last_output_dir, "optimizer.pt"))
    torch.save(scheduler.state_dict(), os.path.join(last_output_dir, "scheduler.pt"))
    print("Saving optimizer and scheduler states to %s", last_output_dir)
#     if _==1:
#         break
#     if (results['f1'] > best_f1):
#         best_f1 = results['f1']
#         #./models/checkpoint-best 检查目录是否存在 否则创建
#         output_dir = os.path.join(args.output_dir, 'checkpoint-best')
#         if not os.path.exists(output_dir):
#             os.makedirs(output_dir)
#         #保存model
# #         model_to_save = model.module if hasattr(model,'module') else model  # Take care of distributed/parallel training
# #         model_to_save.save_pretrained(output_dir)
#         torch.save(model, '%s/cls_model_%d.pth' % (output_dir, epoch))
# #         torch.save(args, os.path.join(output_dir, 'training_{}.bin'.format(idx)))
#         print("Saving model checkpoint to %s", output_dir)
#         #保存优化器的计划器
#         torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
#         torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
#         print("Saving optimizer and scheduler states to %s", output_dir)

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

training!!!
5
loss :0.1485
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
loss :0.0046
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
loss :0.0024
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
loss :0.0019
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
loss :0.0008
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5
5

Epoch:   0%|          | 0/4 [13:13<?, ?it/s]


UnboundLocalError: local variable 'loss' referenced before assignment

In [12]:
checkpoint_prefix = 'checkpoint-best-hdj'
output_dir = os.path.join('/data/hdj/data/CodeBERT/Siamese-model/saved_models/java', '{}'.format(checkpoint_prefix))                        
if not os.path.exists(output_dir):
    os.makedirs(output_dir)         
model_to_save = model.module if hasattr(model,'module') else model
model_to_save.encoder.save_pretrained(output_dir)            
output_dir = os.path.join(output_dir, '{}'.format('model.bin')) 
torch.save(model_to_save.state_dict(), output_dir) 

# 使用cosloss的代码结束

In [14]:
#开始验证模型的有效性 把模型预测结果连同数据一起保存起来 然后再去判断top mrr mAP
#流程 1.训练过程中 每个10轮去在val集上验证 比较f1值是否优于之前，是就保存结果到best，2.每次保存上一次验证的结果 3.最后再在test集上进行测试

#check 词表是否与之前model有很大的不同 如果有那么该怎么改善，之前embedding矩阵是否就作废了
def evaluate(args, model,eval_loader,instances,tokenizer, checkpoint=None, prefix="", mode='dev'):
    '''
        args:模型训练时关键参数
        model：待验证的模型
        eval_loader:待验证的数据集 DataLoader 类型的参数
        instances: 若为test模式，则将原始数据输入进来，然后将预测结果一起保存，等待后面计算TOP MAP MRR使用
        tokenizer:分词器
        checkpoint: 检查点
        prefix:前缀
        mode: 模式 分为 dev 和 test（需要保存数据）
        
        return: dict{'acc':,'f1':,'acc_f1'} 主要关注f1的值 越大说明正例的预测越准
    '''
    #待返回的验证结果
    results={}
    eval_loss = 0.0 #验证损失累加
#     nb_eval_steps = len(test_set)/batch_size #验证批次
    preds = None #保存预测的值
    out_label_ids = None #保存真实标签target
    print('开始验证。。。')
    for _, data in enumerate(eval_loader, 0):#start=0 默认就是0
        with torch.no_grad():
            ids = data[0].to(device, dtype=torch.long).cuda(non_blocking=True)
            mask = data[1].to(device, dtype=torch.long).cuda(non_blocking=True)
            token_type_ids=None#token_type_ids = data[2].to(device, dtype=torch.long).cuda(non_blocking=True) 因为模型不支持2维设置 所以不再输入这个参数
            targets = data[2].to(device, dtype=torch.long).cuda(non_blocking=True)
            if need_feature:
                feature = data[4].to(device, dtype=torch.long).cuda(non_blocking=True)
            if google_v:
                pred = model(input_ids=ids, attention_mask=mask,token_type_ids=None, labels=targets)
                loss=pred[0]
            else:
                if need_feature:
                    pred = model(ids, mask,feature)#输出是[batch_size,2]
                else:
                    pred = model(ids, mask,None)
            # print(pred, target)
#             loss=floss(pred,targets)
            loss = loss_fun(pred, targets)
            pred_choice = pred.max(1)[1]
            correct = pred_choice.eq(targets).cpu().sum()
            metrics = compute_metrics(pred_choice.cpu().numpy(), targets.cpu().numpy())
#             print('[',epoch,': ',_,'/',nb_eval_steps,'] ',blue('val')," loss :%.4f" % loss.item(),' , accuracy: ',correct.item() / float(batch_size) ,'true nums:',sum(targets.cpu().numpy()),' ration :',sum(targets.cpu().numpy())/len(targets),"acc :",metrics['acc']," f1 :",metrics['f1'])
#             loss_list_val.append(loss.item())  
            global eval_step
            tb_writer.add_scalar('eval_loss', loss.item(), eval_step)
            tb_writer.add_scalar('eval_acc', correct.item() / float(batch_size), eval_step)
            tb_writer.add_scalar('eval_f1', metrics['f1'], eval_step)
            eval_step+=1
#             print(eval_step)
            eval_loss += loss.item()
            if(_%100==0):
                print(_,'/',len(eval_loader),loss)
#         nb_eval_steps += 1
        #将preds值和标签进行保存
        if preds is None:
            if google_v:
                preds = pred[1].detach().cpu()
            else:
                preds=pred.detach().cpu()
            out_label_ids = targets.detach().cpu()
        else:
            if google_v:
                preds = np.append(preds, pred[1].detach().cpu().numpy(), axis=0)
            else:    
                preds = np.append(preds, pred.detach().cpu().numpy(), axis=0)
            out_label_ids = np.append(out_label_ids, targets.detach().cpu().numpy(), axis=0)       
    print('验证结束，开始计算指标',eval_loss)
    #计算指标，并更新结果
 
    #ndarra转换为tensor类型
    preds=torch.tensor(preds)
    out_label_ids=torch.tensor(out_label_ids)
#     print(type(preds),preds.shape,preds)
#     print(type(out_label_ids),out_label_ids.shape,out_label_ids)    
    pred_choice = preds.max(1)[1]#[batch_size,2]
    correct = pred_choice.eq(out_label_ids).cpu().sum() #eq需要tensor类型的数据
    #但是compute_metrics 需要numpy的数据类型 真烦人
    metrics = compute_metrics(pred_choice.numpy(), out_label_ids.numpy())
    results.update(metrics)
    #计算完成
    
    #下面分dev 和 test 模式来完成数据模型的保存工作
    if(mode=='dev'):
        #这里只是把result字典里acc和f1给保存到文件
        output_eval_file = os.path.join(args.output_dir, "eval_results.txt")
        with open(output_eval_file, "a+") as writer:
            print("***** Eval results {} *****".format(prefix))
            writer.write('evaluate %s\n' % checkpoint)
            for key in sorted(results.keys()):
                writer.write("%s = %s\n" % (key, str(results[key])))
    elif(mode=='test'):
        #test结果目录 将test数据连同预测结果进行保存
        if(instances==None or len(instances)==0):
            print('输入的测试数据有问题 为None 或者 长度为0')
        #如果目录不存在就创建
        output_test_file = args.test_result_dir
        output_dir = os.path.dirname(output_test_file)
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        #检验创建目录完成
        with open(output_test_file, "w") as writer:
            print("***** Output test results *****")
            preds = F.softmax(torch.tensor(preds))
            all_logits = preds.tolist()
            for i, logit in tqdm(enumerate(all_logits), desc='Testing'):#desc是进度条的标题
                #instances即代表test集里每一行数据的值
                instance_rep = '<CODESPLIT>'.join([item.encode('ascii', 'ignore').decode('ascii') for item in instances[i]])
                writer.write(instance_rep + '<CODESPLIT>' + '<CODESPLIT>'.join([str(l) for l in logit]) + '\n')
            #打印验证的结果
            for key in sorted(results.keys()):
                print("%s = %s" % (key, str(results[key])))
    return results

In [14]:
import gc
gc.collect()

120

In [15]:
#模型应该保存效果最好的那个，而不是训练到最后的那个模型
print("training!!!")
blue = lambda x: '\033[94m' + x + '\033[0m'
epoch=1
best_f1=0.0 #保存最好的f1的模型
model.zero_grad()
set_seed(args)  # Added here for reproductibility (even between python 2 and 3)
#这里就是在指定 一共训练多少Epoch [start_epoch,num_train_epcchs]
t_total = len(training_loader) // args.gradient_accumulation_steps * args.num_train_epochs
scheduler = get_linear_schedule_with_warmup(optimizer, args.warmup_steps, t_total)
global_step=0
eval_step=0
train_iterator = trange(args.start_epoch, int(args.num_train_epochs), desc="Epoch")
model.train()
for idx, p in enumerate(train_iterator):
    loss_list = []
    acc_list = []
    loss_list_val = []
    acc_list_val = []
    num_batch = len(train_set) / batch_size #总共训练次数
    for _, data in enumerate(training_loader, 0):#start=0 默认就是0
        ids = data[0].to(device, dtype=torch.long).cuda(non_blocking=True)
        mask = data[1].to(device, dtype=torch.long).cuda(non_blocking=True)
        token_type_ids=None
    #     token_type_ids = data[2].to(device, dtype=torch.long).cuda(non_blocking=True) 因为模型不支持2维设置 所以不再输入这个参数
        targets = data[2].to(device, dtype=torch.long).cuda(non_blocking=True)
        if need_feature:
            feature=data[4].to(device, dtype=torch.long).cuda(non_blocking=True)
#         print('feature shape: ',feature.shape)
#         print(feature)
        if google_v:
#             outputs = model(ids, mask, token_type_ids)
            outputs = model(input_ids=ids, attention_mask=mask,token_type_ids=None, labels=targets)
            loss=outputs[0]#google loss
#             print(outputs)
#             loss_hdj = loss_fun(outputs[1], targets)
#             print('google loss ',loss,' loss_hdj :',loss_hdj)
            pred_choice = outputs[1].max(1)[1]
        else:
            if need_feature:
                outputs = model(ids, mask,feature)#输出是[batch_size,2]
            else:
                outputs = model(ids, mask,None)
            #使用floss
#             loss=floss(outputs,targets)
            #使用floss
            #使用ghmloss
#             ghm_target = targets.view(-1, 1)
#             ghm_target=ghm_target.cpu()
#             ghm_target = torch.LongTensor(ghm_target)
#             # print("before target :",targets)
#             ghm_target = torch.zeros(len(ghm_target), 2).scatter_(1, ghm_target, 1)
#             outputs=outputs.cpu()
#             loss=ghm(outputs,ghm_target)
            #使用ghmloss
            # print(outputs)
            
            #交叉熵loss
            loss = loss_fun(outputs, targets)
            #交叉熵loss
    
    #         print('查看target和ouput的类型 检查max函数返回的东西是否正确',type(outputs),type(targets))
        #     print(outputs)
            #取outputs二维中最大维的下标
            pred_choice = outputs.max(1)[1]
        if args.gradient_accumulation_steps > 1:
             loss = loss / args.gradient_accumulation_steps
                
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), args.max_grad_norm)
        
#         targets=targets.cpu()
        correct = pred_choice.eq(targets).cpu().sum()
        metrics = compute_metrics(pred_choice.cpu().numpy(), targets.cpu().numpy())
        if _ %100 == 0:
            print('[',epoch,':',_,'/',num_batch,']',"loss :%.4f" % loss.item(),' , accuracy: ',correct.item() / float(batch_size) ,'true nums:',sum(targets.cpu().numpy()),' ration :',sum(targets.cpu().numpy())/len(targets),"acc :",metrics['acc']," f1 :",metrics['f1'])
#         loss_list.append(loss.item())
#         acc_list.append(correct.item() / float(batch_size))
        if(_+1)%args.gradient_accumulation_steps == 0:
            global_step+=1
            optimizer.step()
            scheduler.step()
            model.zero_grad()
            tb_writer.add_scalar('train_loss', loss.item(), global_step)
            tb_writer.add_scalar('train_acc', correct.item() / float(batch_size), global_step)
            tb_writer.add_scalar('train_f1', metrics['f1'], global_step)
    #下面是在训练过程中进行验证 每一轮都会验证一次，保存模型最好的 和最近一次的模型
    results = evaluate(args, model,val_loader,None ,tokenizer, checkpoint=str(args.start_epoch + idx),mode='dev')
    print('验证结果 ：',results)
    #torch.save(model, '%s/cls_model_%d.pth' % ('models', epoch))
    #./models/java/checkpoint-last
    last_output_dir = os.path.join(args.output_dir, 'checkpoint-last')
    if not os.path.exists(last_output_dir):
        os.makedirs(last_output_dir)
#     model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
#     model_to_save.save_pretrained(last_output_dir)
    torch.save(model, '%s/cls_model_%d.pth' % (last_output_dir, epoch))
    print("Saving model checkpoint to %s", last_output_dir)
    #保存idx_file 文件 第几轮训练的记录
    idx_file = os.path.join(last_output_dir, 'idx_file.txt')
    with open(idx_file, 'w', encoding='utf-8') as idxf:
        idxf.write(str(args.start_epoch + idx) + '\n')
    #保存idx_file完成
    #保存优化器和计划器
    torch.save(optimizer.state_dict(), os.path.join(last_output_dir, "optimizer.pt"))
    torch.save(scheduler.state_dict(), os.path.join(last_output_dir, "scheduler.pt"))
    print("Saving optimizer and scheduler states to %s", last_output_dir)
#     if _==1:
#         break
    if (results['f1'] > best_f1):
        best_f1 = results['f1']
        #./models/checkpoint-best 检查目录是否存在 否则创建
        output_dir = os.path.join(args.output_dir, 'checkpoint-best')
        if not os.path.exists(output_dir):
            os.makedirs(output_dir)
        #保存model
#         model_to_save = model.module if hasattr(model,'module') else model  # Take care of distributed/parallel training
#         model_to_save.save_pretrained(output_dir)
        torch.save(model, '%s/cls_model_%d.pth' % (output_dir, epoch))
#         torch.save(args, os.path.join(output_dir, 'training_{}.bin'.format(idx)))
        print("Saving model checkpoint to %s", output_dir)
        #保存优化器的计划器
        torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
        torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
        print("Saving optimizer and scheduler states to %s", output_dir)

Epoch:   0%|          | 0/4 [00:00<?, ?it/s]

training!!!


Epoch:   0%|          | 0/4 [00:00<?, ?it/s]


TypeError: forward() takes from 1 to 3 positional arguments but 4 were given

In [63]:
# loss_list

In [49]:
1

1

In [ ]:
torch.Size([64, 100, 40, 100]) torch.Size([64, 208, 100])
code第一次拼接的shape torch.Size([6400, 300, 1, 1])
code第一次卷积 torch.Size([192, 1, 100, 100])
torch.Size([192, 300, 1, 1])
code size report size :  torch.Size([192, 300]) torch.Size([64, 300])

In [10]:
class GHM_Loss(nn.Module):
    def __init__(self, bins, alpha):
        super(GHM_Loss, self).__init__()
        self._bins = bins
        self._alpha = alpha
        self._last_bin_count = None

    def _g2bin(self, g):
        return torch.floor(g * (self._bins - 0.0001)).long()

    def _custom_loss(self, x, target, weight):
        raise NotImplementedError

    def _custom_loss_grad(self, x, target):
        raise NotImplementedError

    def forward(self, x, target):
        g = torch.abs(self._custom_loss_grad(x, target)).detach()

        bin_idx = self._g2bin(g)

        bin_count = torch.zeros((self._bins))
        for i in range(self._bins):
            bin_count[i] = (bin_idx == i).sum().item()

        N = (x.size(0) * x.size(1))

        if self._last_bin_count is None:
            self._last_bin_count = bin_count
        else:
            bin_count = self._alpha * self._last_bin_count + (1 - self._alpha) * bin_count
            self._last_bin_count = bin_count

        nonempty_bins = (bin_count > 0).sum().item()

        gd = bin_count * nonempty_bins
        gd = torch.clamp(gd, min=0.0001)
        beta = N / gd

        return self._custom_loss(x, target, beta[bin_idx])

class GHMC_Loss(GHM_Loss):
    def __init__(self, bins, alpha):
        super(GHMC_Loss, self).__init__(bins, alpha)

    def _custom_loss(self, x, target, weight):
        return F.binary_cross_entropy_with_logits(x, target, weight=weight)

    def _custom_loss_grad(self, x, target):
        return torch.sigmoid(x).detach() - target
ghm=GHMC_Loss(bins=30,alpha=0.75)

In [10]:
from torch import nn
import torch
from torch.nn import functional as F

class focal_loss(nn.Module):
    def __init__(self, alpha=[0.25,0.75], gamma=2, num_classes = 2, size_average=False):
        """
        focal_loss损失函数, -α(1-yi)**γ *ce_loss(xi,yi)
        步骤详细的实现了 focal_loss损失函数.
        :param alpha: 阿尔法α,类别权重.      
                    当α是列表时,为各类别权重；
                    当α为常数时,类别权重为[α, 1-α, 1-α, ....],
                    常用于目标检测算法中抑制背景类, 
                    retainnet中设置为0.25
        :param gamma: 伽马γ,难易样本调节参数. retainnet中设置为2
        :param num_classes: 类别数量
        :param size_average: 损失计算方式,默认取均值
        """
        super(focal_loss,self).__init__()
        self.size_average = size_average
        if isinstance(alpha,list):
            assert len(alpha)==num_classes   
            # α可以以list方式输入,
            # size:[num_classes] 用于对不同类别精细地赋予权重
            print("Focal_loss alpha = {}, 将对每一类权重进行精细化赋值".format(alpha))
            self.alpha = torch.Tensor(alpha)
        else:
            assert alpha<1  #如果α为一个常数,则降低第一类的影响,在目标检测中为第一类
            print("Focal_loss alpha = {} ,将对背景类进行衰减,请在目标检测任务中使用.".format(alpha))
            self.alpha = torch.zeros(num_classes)
            self.alpha[0] += alpha
            self.alpha[1:] += (1-alpha) # α 最终为 [ α, 1-α, 1-α, 1-α, 1-α, ...] size:[num_classes]

        self.gamma = gamma

    def forward(self, preds, labels):
        """
        focal_loss损失计算
        :param preds: 预测类别. size:[B,N,C] or [B,C]    分
                别对应与检测与分类任务, B 批次, N检测框数, C类别数
        :param labels:  实际类别. size:[B,N] or [B]
        :return:
        """
        # assert preds.dim()==2 and labels.dim()==1
        preds = preds.view(-1,preds.size(-1))
#         print('preds :',preds)
        self.alpha = self.alpha.to(preds.device)
        # 这里并没有直接使用log_softmax, 因为后面会用到softmax的结果(当然也可以使用log_softmax,然后进行exp操作)
        preds_softmax = F.softmax(preds, dim=1) 
#         print('preds_softmax :',preds_softmax)
        preds_logsoft = torch.log(preds_softmax)
#         print('preds_logsoft :',preds_logsoft)        
        # 这部分实现nll_loss ( crossempty = log_softmax + nll )
        preds_softmax = preds_softmax.gather(1,labels.view(-1,1))   
#         print('preds_softmax :',preds_softmax) 
        preds_logsoft = preds_logsoft.gather(1,labels.view(-1,1))
#         print('preds_logsoft :',preds_logsoft) 
        self.alpha = self.alpha.gather(0,labels.view(-1))
        loss = -torch.mul(torch.pow((1-preds_softmax), self.gamma), preds_logsoft)  
        # torch.pow((1-preds_softmax), self.gamma) 为focal loss中 (1-pt)**γ

        loss = torch.mul(self.alpha, loss.t())
        if self.size_average:
            loss = loss.mean()
        else:
            loss = loss.sum()
        return loss
floss=focal_loss()

Focal_loss alpha = [0.25, 0.75], 将对每一类权重进行精细化赋值


In [11]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class FocalLoss(nn.Module):
    def __init__(self, alpha=[0.25,0.75], gamma=2, size_average=True):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = torch.Tensor([gamma])
        self.size_average = size_average
        if isinstance(alpha, (float, int)):
            if self.alpha > 1:
                raise ValueError('Not supported value, alpha should be small than 1.0')
            else:
                self.alpha = torch.Tensor([alpha, 1.0 - alpha])
        if isinstance(alpha, list): self.alpha = torch.Tensor(alpha)
        self.alpha /= torch.sum(self.alpha)

    def forward(self, input, target):
        if input.dim() > 2:
            input = input.view(input.size(0), input.size(1), -1)  # [N,C,H,W]->[N,C,H*W] ([N,C,D,H,W]->[N,C,D*H*W])
        # target
        # [N,1,D,H,W] ->[N*D*H*W,1]
        if self.alpha.device != input.device:
            self.alpha = torch.tensor(self.alpha, device=input.device)
        target = target.view(-1, 1)
        logpt = torch.log(input + 1e-10)
        logpt = logpt.gather(1, target)
        logpt = logpt.view(-1, 1)
        pt = torch.exp(logpt)
        alpha = self.alpha.gather(0, target.view(-1))

        gamma = self.gamma

        if not self.gamma.device == input.device:
            gamma = torch.tensor(self.gamma, device=input.device)

        loss = -1 * alpha * torch.pow((1 - pt), gamma) * logpt
        if self.size_average:
            loss = loss.mean()
        else:
            loss = loss.sum()
        return loss
floss=FocalLoss()

# 训练和验证并保存最好模型

In [23]:
# #下面是在训练过程中进行验证 每一轮都会验证一次，保存模型最好的 和最近一次的模型
# results = evaluate(args, model,val_loader,None ,tokenizer, checkpoint=str(args.start_epoch + idx),mode='dev')
# print('验证结果 ：',results)
# #torch.save(model, '%s/cls_model_%d.pth' % ('models', epoch))
# #./models/java/checkpoint-last
# last_output_dir = os.path.join(args.output_dir, 'checkpoint-last')
# if not os.path.exists(last_output_dir):
#     os.makedirs(last_output_dir)
# #     model_to_save = model.module if hasattr(model, 'module') else model  # Take care of distributed/parallel training
# #     model_to_save.save_pretrained(last_output_dir)
# torch.save(model, '%s/cls_model_%d.pth' % (last_output_dir, epoch))
# print("Saving model checkpoint to %s", last_output_dir)
# #保存idx_file 文件 第几轮训练的记录
# idx_file = os.path.join(last_output_dir, 'idx_file.txt')
# with open(idx_file, 'w', encoding='utf-8') as idxf:
#     idxf.write(str(args.start_epoch + idx) + '\n')
# #保存idx_file完成
# #保存优化器和计划器
# torch.save(optimizer.state_dict(), os.path.join(last_output_dir, "optimizer.pt"))
# torch.save(scheduler.state_dict(), os.path.join(last_output_dir, "scheduler.pt"))
# print("Saving optimizer and scheduler states to %s", last_output_dir)

# if (results['f1'] > best_f1):
#     best_f1 = results['f1']
#     #./models/checkpoint-best 检查目录是否存在 否则创建
#     output_dir = os.path.join(args.output_dir, 'checkpoint-best')
#     if not os.path.exists(output_dir):
#         os.makedirs(output_dir)
#     #保存model
# #         model_to_save = model.module if hasattr(model,'module') else model  # Take care of distributed/parallel training
# #         model_to_save.save_pretrained(output_dir)
#         torch.save(model, '%s/cls_model_%d.pth' % (output_dir, epoch))
# #         torch.save(args, os.path.join(output_dir, 'training_{}.bin'.format(idx)))
#     print("Saving model checkpoint to %s", output_dir)
#     #保存优化器的计划器
#     torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
#     torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
#     print("Saving optimizer and scheduler states to %s", output_dir)

# 加载模型进行test

In [7]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
google_v=False #当为true时，使用RobertClassification模型
num_labels=2
batch_size=64

In [14]:
# results = evaluate(args, model,val_loader,None ,tokenizer, checkpoint=str(args.start_epoch + idx),mode='dev')
#下面开启test模型性能
#加载模型进行测试
# del model
import gc
gc.collect()
# file_prefix=''
# train_file=''
# dev_file=''
# task_name='codesearch'
# max_seq_length=256
test_dir="/data/hdj/tracking_buggy_files/joblib_memmap_"+project+"/one_vs_two/" #注意目前测试集的目录是单独的，更换测试数据需要更改
# test_dir='/data/hdj/data/CodeBERT/data/codesearch/test/zxing_test/zxing/'
# test_file = "test_all.txt"#目前只使用一个例子来进行测试，后期增加数据量
# test_dir="/data/hdj/data/CodeBERT/data/codesearch/test/swt_test/swt/" 
test_file = "test_2hard_2_source_nomethod_less.txt"#
# train_file = "aspectj_train_small.txt"
# dev_file = "aspectj_val_small.txt"
eval_batch_size=64
test_set, instances=load_and_cache_examples_hdj(file_prefix,test_dir,train_file,dev_file,test_file,task_name,tokenizer,max_seq_length,'test')
testing_loader = DataLoader(dataset=test_set, batch_size=32, shuffle=False, num_workers=2, drop_last=False)

直接加载cache文件
cached_features_file : /data/hdj/tracking_buggy_files/joblib_memmap_eclipse/one_vs_two/cached_test_test_2hard_2_source_nomethod_less_codebert-base_512_codesearch
[Errno 2] No such file or directory: '/data/hdj/tracking_buggy_files/joblib_memmap_eclipse/one_vs_two/cached_test_test_2hard_2_source_nomethod_less_codebert-base_512_codesearch'
生成cache文件
Writing example 0 of 30000
Writing example 10000 of 30000
Writing example 20000 of 30000


In [14]:
testing_loader = DataLoader(dataset=test_set, batch_size=64, shuffle=False, num_workers=2, drop_last=False)

In [33]:
# testing_loader.dataset.tensors[0].shape
print('测试集比例',sum(list(testing_loader.dataset.tensors[3])),len(testing_loader.dataset.tensors[3]),sum(list(testing_loader.dataset.tensors[3]))/len(testing_loader.dataset.tensors[3]))

测试集比例 tensor(815) 30000 tensor(0.0272)


In [15]:
#先不删除模型直接进行预测 之后再删除模型 加载保存下来的模型进行预测
test_results = evaluate(args, model,testing_loader,instances ,tokenizer, checkpoint=str(args.start_epoch + idx),mode='test')

开始验证。。。
0 / 938 tensor(0.7061, device='cuda:0')
100 / 938 tensor(1.0171, device='cuda:0')
200 / 938 tensor(0.9639, device='cuda:0')
300 / 938 tensor(1.0116, device='cuda:0')
400 / 938 tensor(0.8404, device='cuda:0')
500 / 938 tensor(0.9830, device='cuda:0')
600 / 938 tensor(1.0427, device='cuda:0')
700 / 938 tensor(0.8450, device='cuda:0')
800 / 938 tensor(0.5988, device='cuda:0')
900 / 938 tensor(0.7327, device='cuda:0')


/home/hdj/anaconda3/envs/waf/lib/python3.7/site-packages/ipykernel_launcher.py:108: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/hdj/anaconda3/envs/waf/lib/python3.7/site-packages/ipykernel_launcher.py:108: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
Testing: 0it [00:00, ?it/s]

验证结束，开始计算指标 823.756529957056
***** Output test results *****


Testing: 30000it [00:00, 109200.88it/s]

acc = 0.5598333333333333
acc_and_f1 = 0.30454046967624254
f1 = 0.04924760601915184


In [20]:
#加载模型 进行验证 保存结果 22;43
del model
import gc
gc.collect()
device = 'cuda' if torch.cuda.is_available() else 'cpu'
need_feature=False
epoch=1
idx=1
eval_step=0
google_v=False #当为true时，使用RobertClassification模型
num_labels=2
batch_size=512
output_dir = os.path.join(args.output_dir, 'checkpoint-best')
# output_dir = os.path.join(args.output_dir, 'checkpoint-last')
model=torch.load('%s/cls_model_%d.pth' % (output_dir, epoch))
test_results = evaluate(args, model,testing_loader,instances ,tokenizer, checkpoint=str(args.start_epoch + idx),mode='test')

开始验证。。。
0 / 156 tensor(0.0597, device='cuda:0')
100 / 156 tensor(0.6250, device='cuda:0')


/home/hdj/anaconda3/envs/waf/lib/python3.7/site-packages/ipykernel_launcher.py:108: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/home/hdj/anaconda3/envs/waf/lib/python3.7/site-packages/ipykernel_launcher.py:108: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
Testing: 0it [00:00, ?it/s]

验证结束，开始计算指标 256.23298240453005
***** Output test results *****


Testing: 79415it [00:00, 85967.44it/s]


acc = 0.67504879430838
acc_and_f1 = 0.39524702689299857
f1 = 0.11544525947761707


In [31]:
# #使用codeBert的模型不微调 测试效果如何
# idx=0
# test_results = evaluate(args, model,testing_loader,instances ,tokenizer, checkpoint=str(args.start_epoch + idx),mode='test')

# 计算最终指标

In [19]:
#开始计算TOP MRR MAP指标
#有个问题 如果预测a=[0.2,0.5] b=[0.8,0.7] 这样的话，是将a排在b前 还是相反 ；暂时还是按照分数的绝对大小来判断吧
#每1000验证一次三个指标
# path='/data/hdj/data/CodeBERT/codesearch/results/java/aspectj_withou_code_clasMeth_summDesc_result_all.txt'
# path='/data/hdj/data/CodeBERT/codesearch/results/java/aspectj_clasMeth_summDesc_result_all.txt'
# path='/data/hdj/data/CodeBERT/codesearch/results/java/aspectj_oversam_60_clasMeth_summDesc_result_all.txt'

# path='/data/hdj/data/CodeBERT/codesearch/results/java/swt_clasMeth_summDesc_result_all.txt'
# path='/data/hdj/data/CodeBERT/codesearch/results/java/swt_overoversam_60_clasMeth_summDesc_result_all.txt'
# path='/data/hdj/data/CodeBERT/codesearch/results/java/swt_withou_code_clasMeth_summDesc_result_all.txt'

# path='/data/hdj/data/CodeBERT/codesearch/results/java/512_0202_result_all.txt'
# path='/data/hdj/data/CodeBERT/codesearch/results/java/result_all.txt'
path='/data/hdj/data/CodeBERT/codesearch/results/java/test_v1_0414.txt'
# path='/data/hdj/data/CodeBERT/codesearch/results/java/0415_aspectj_withou_code_clasMeth_summDesc_result_all.txt'
# path='/data/hdj/data/CodeBERT/codesearch/results/java/0415_swt_withou_code_clasMeth_summDesc_result_all.txt'
with open(path,'r',encoding='utf-8') as f_in:
    data = f_in.readlines()
    print('读取的文件长度 :',len(data))
    #注意swt是479 aspectj是1000 zxing是390
#     batched_data = chunked(data, 479)
#     batched_data = chunked(data, 390)
    batched_data = chunked(data, 300)
    print("start processing")
    top10ranks=[]
    MRRranks=[]
    MAPranks=[]
    for batch_idx, batch_data in enumerate(batched_data):
        preds=[]
        out_label_ids=[]
        report_ids=[]
        paths=[]
        for d_idx, d in enumerate(batch_data):
            line=d.split('<CODESPLIT>')
            out_label_ids.append(int(line[0]))
            preds.append(float(line[-1]))
            report_ids.append(line[1])
            paths.append(line[2])
        # print(len(preds),len(out_label_ids))
        top10rank, MRRrank, MAPrank = eval_mrr(preds, out_label_ids)
        print('top10rank, MRRrank, MAPrank',top10rank, MRRrank, MAPrank)
        top10ranks.append(top10rank)
        MRRranks.append(MRRrank)
        MAPranks.append(MAPrank)
        # with open('/data/hdj/data/CodeBERT/data/codesearch/test/aspectj_result_check/aspectj_'+str(batch_idx)+'.txt','w',encoding='utf-8') as f_out:
        #     # f_out.writelines('\n'.join(out_label_ids))
        #      for label,pred,id,path in zip(out_label_ids,preds,report_ids,paths):
        #          f_out.write(str(label)+" "+str(pred)+' '+id+' '+path+'\n')

    print('最后平均得分 top10rank, MRRrank, MAPrank :', sum(top10ranks)/len(top10ranks), sum(MRRranks)/len(MRRranks),sum(MAPranks)/len(MAPranks) )

读取的文件长度 : 17400
start processing
top10rank, MRRrank, MAPrank 0 0.004184173669467787 0.006267507002801121
top10rank, MRRrank, MAPrank 0 0.005557720717974844 0.01454896650276409
top10rank, MRRrank, MAPrank 0 0.006756756756756757 0.006756756756756757
top10rank, MRRrank, MAPrank 0 0.004200427935038059 0.00613841243116209
top10rank, MRRrank, MAPrank 0 0.005180272566285254 0.011882916053344916
top10rank, MRRrank, MAPrank 0 0.005190282381195985 0.011230199928064873
top10rank, MRRrank, MAPrank 0 0.003663003663003663 0.003663003663003663
top10rank, MRRrank, MAPrank 0 0.005494505494505495 0.005494505494505495
top10rank, MRRrank, MAPrank 0 0.023278029160382102 0.03863750628456511
top10rank, MRRrank, MAPrank 0 0.003745318352059925 0.003745318352059925
top10rank, MRRrank, MAPrank 0 0.005045895939366729 0.009100515114260818
top10rank, MRRrank, MAPrank 0 0.013069818133109272 0.02017523030181258
top10rank, MRRrank, MAPrank 0 0.009566885964912281 0.011321271929824562
top10rank, MRRrank, MAPrank 0 0.005

In [47]:
37362/479

78.0

# 下面试测试功能的代码

In [8]:
#测试compute_metrics函数作用的代码
import numpy as np
from sklearn.metrics import f1_score
#从这来看，必须保证正例样本要有，不然f1值都为0 尽量使数据不要那么失衡
logit_last=np.array([[4,5],[4,5],[3,2]])
label_ids=np.array([1,1,1])
preds_label_last = np.argmax(logit_last, axis=1)
result_last = compute_metrics(preds_label_last, label_ids)
print(result_last['acc'],result_last['f1'])
#测试结束

0.6666666666666666 0.8


In [9]:
#测试字典更新函数的代码
results={}
res1={'acc':0.5,'f1':0.6}
res2={'acc':0.6,'f1':0.7}
results.update(res1)
print(results)
results.update(res2)
print(results)
#测试结束

{'acc': 0.5, 'f1': 0.6}
{'acc': 0.6, 'f1': 0.7}


In [154]:
#测试shuffle作用的代码
all_input_ids=torch.tensor([[1,1,1],[2,2,2],[3,3,3],[4,4,4],[5,5,5],[6,6,6]])
dataset = TensorDataset(all_input_ids)
testing_loader = DataLoader(dataset=dataset, batch_size=4, shuffle=False, num_workers=2, drop_last=False)
testing_loader.dataset.tensors
for i, data in enumerate(testing_loader, 0):
    print(i, "inputs", data)
#测试结束

0 inputs [tensor([[1, 1, 1],
        [2, 2, 2],
        [3, 3, 3],
        [4, 4, 4]])]
1 inputs [tensor([[5, 5, 5],
        [6, 6, 6]])]


In [ ]:
#测试eq函数的代码 eq只能是tensor变量才能使用
preds=torch.tensor(np.array([[2.02,2.00],[1.26,1],[-2.3,1.2]]))
print(type(preds),preds)
targets=torch.tensor([0,1,1])
pred_choice = preds.max(1)[1]#[batch_size,2]
print(type(pred_choice),pred_choice)
correct = pred_choice.eq(targets).cpu().sum()
correct
#测试结束

In [27]:
#测试trange如何使用
from tqdm import tqdm, trange
# #这里就是在指定 一共训练多少Epoch
# train_iterator = trange(args.start_epoch, int(args.num_train_epochs), desc="Epoch")
# for idx, _ in enumerate(train_iterator):
#     print("a",idx,_)

In [29]:
#测试weight参数如何设置
#输入target 一维类别LongTensor input二维 增加weight来控制0类别的比例
weights=torch.tensor([0.2,0.7]).cuda()
loss_fun=CrossEntropyLoss(weight=weights)
# pred = torch.FloatTensor([[0.5, 0.8], [-0.1, 0.7]]).cuda()
# target = torch.LongTensor([1, 0]).cuda()
# print(pred)
# loss=loss_fun(pred,target)
# loss

In [3]:
# # 查看测试文件的顺序是否一致 必须一致下面计算TOP MRR MAP的结果才能正确
# with open('/data/hdj/data/CodeBERT/codesearch/results/java/swt_clasMeth_summDesc_result_all.txt','r',encoding='utf-8') as f_in:
#     for i,line  in enumerate(f_in):
#         strings=line.split('<CODESPLIT>')
#         print(strings[0],strings[1],strings[-2],strings[-1])
#         if(i==2000):
#             break
# # 查看test集的数据是否正常
# for i,ins in  enumerate(instances):
#     print(i,ins[0],ins[1])


In [ ]:
#加载打印模型的相关信息
# from transformers import AutoModel
# tokenizer=RobertaTokenizer.from_pretrained('microsoft/codebert-base')
# tokenizer.cls_token,tokenizer.cls_token_id,tokenizer.sep_token,tokenizer.sep_token_id

In [6]:
class args(object):
    """A single set of features of data."""

    def __init__(self):
        self.model_type = 'roberta'
        self.output_dir='/data/hdj/data/CodeBERT/codesearch/models/java'
        self.test_result_dir='/data/hdj/data/CodeBERT/codesearch/results/java/test_v1_0414.txt'
        self.start_epoch=0
        self.num_train_epochs=1
        self.model_type='roberta'
        self.config_name=''
        self.model_name_or_path=None
        self.task_name='codesearch'
        self.tokenizer_name=''
        self.model_name_or_path='microsoft/codebert-base'
        self.do_lower_case=True
args=args()
# weights=torch.tensor([0.1,0.9]).cuda()
# loss_fun=CrossEntropyLoss(weight=weights)
# loss_fun=CrossEntropyLoss()

In [28]:
#210415 问题1 RobertClassification模型具体结构和参数使用
num_labels=2
MODEL_CLASSES = {'roberta': (RobertaConfig, RobertaForSequenceClassification, RobertaTokenizer)}
config_class, model_class, tokenizer_class = MODEL_CLASSES[args.model_type]
config = config_class.from_pretrained(args.config_name if args.config_name else args.model_name_or_path,
                                      num_labels=num_labels, finetuning_task=args.task_name)
if args.tokenizer_name:
    tokenizer_name = args.tokenizer_name
elif args.model_name_or_path:
    tokenizer_name = 'roberta-base'
tokenizer = tokenizer_class.from_pretrained(tokenizer_name, do_lower_case=args.do_lower_case)
model = model_class.from_pretrained(args.model_name_or_path, from_tf=bool('.ckpt' in args.model_name_or_path),
                                    config=config)

In [29]:
# config
input_ids=torch.tensor([[1,1,1],[2,2,2],[3,3,3]])
attention_mask=torch.tensor([[1,1,1],[1,1,1],[1,1,1]])
label=torch.tensor([0,0,1])


In [30]:
output = model(input_ids=input_ids, attention_mask=attention_mask,token_type_ids=None, labels=label)

In [31]:
output

(tensor(0.7257, grad_fn=<NllLossBackward>),
 tensor([[-0.0580,  0.1690],
         [ 0.0895,  0.0964],
         [ 0.0569,  0.1093]], grad_fn=<AddmmBackward>))

In [32]:
y_pred_prob = output[1]
y_pred_label = y_pred_prob.argmax(dim=1)
# 计算loss
# 这个 loss 和 output[0] 是一样的
criterion = nn.CrossEntropyLoss()
loss = criterion(y_pred_prob.view(-1, 2), label.view(-1))

In [33]:
# print(model)
loss

tensor(0.7257, grad_fn=<NllLossBackward>)

In [24]:
#可以覆盖最后一层的值
# model.classifier.out_proj=nn.Linear(768, 9)
model.base_model.add_module=nn.Linear(768,768)
model.base_model.add_module=nn.Linear(768,768)
model.base_model.

In [26]:
robertaModel=RobertaModel.from_pretrained("microsoft/codebert-base")

In [1]:
# print(robertaModel)

In [4]:
import pandas as pd
aspectj_20_training_neg=pd.read_csv('/data/hdj/cross_project_trans/report/aspectj_20_training_neg.csv')


,id,path,label
0,146546,org.aspectj/modules/aspectj-attic/ajdoc-src/or...,0
1,146546,org.aspectj/modules/docs/sandbox/ubc-design-pa...,0
2,146546,org.aspectj/modules/tests/bugs150/pr115252/Exa...,0
3,146546,org.aspectj/modules/bcel-builder/src/org/aspec...,0
4,146546,org.aspectj/modules/tests/ajde/examples/spacew...,0
5,146546,org.aspectj/modules/tests/pureJava/BoundaryNum...,0
6,146546,org.aspectj/modules/docs/sandbox/ubc-design-pa...,0
7,146546,org.aspectj/modules/tests/new/StaticIntroduced...,0
8,146546,org.aspectj/modules/tests/bugs/IntertypeDiffer...,0
9,146546,org.aspectj/modules/tests/bugs153/pr153490/Foo...,0


In [14]:
len(set(aspectj_20_training_neg['id']))

58